In [3]:
import torch

In [12]:
pth_path = "../../Data/BNN_weight/for quantize_3layer_256_89.pth"
# pth_path = "../../Data/NN_weight/LeNet5_MNIST.pth"
ckpt = torch.load(pth_path, map_location="cpu", weights_only=True)
state = ckpt.get("state_dict", ckpt)  # 若含 state_dict 则取出

for k, v in state.items():
    print(f"{k:30} → shape: {tuple(v.shape)}")

conv_entry.weight              → shape: (32, 3, 3, 3)
conv_entry.bias                → shape: (32,)
bn_entry.weight                → shape: (32,)
bn_entry.bias                  → shape: (32,)
bn_entry.running_mean          → shape: (32,)
bn_entry.running_var           → shape: (32,)
bn_entry.num_batches_tracked   → shape: ()
dsc_conv1_1.depth_conv.weight  → shape: (32, 1, 3, 3)
dsc_conv1_1.depth_conv.bias    → shape: (32,)
dsc_conv1_1.depth_bn.weight    → shape: (32,)
dsc_conv1_1.depth_bn.bias      → shape: (32,)
dsc_conv1_1.depth_bn.running_mean → shape: (32,)
dsc_conv1_1.depth_bn.running_var → shape: (32,)
dsc_conv1_1.depth_bn.num_batches_tracked → shape: ()
dsc_conv1_1.point_conv.weight  → shape: (64, 32, 1, 1)
dsc_conv1_1.point_conv.bias    → shape: (64,)
dsc_conv1_1.point_bn.weight    → shape: (64,)
dsc_conv1_1.point_bn.bias      → shape: (64,)
dsc_conv1_1.point_bn.running_mean → shape: (64,)
dsc_conv1_1.point_bn.running_var → shape: (64,)
dsc_conv1_1.point_bn.num_batches_tracked 